In [1]:
from textwrap import dedent
from typing import Any, Dict, List, Union

from neollm import MyLLM
from neollm.utils.preprocess import dict2json
from neollm.utils.postprocess import json2dict

In [43]:
import gradio as gr


class ChatBotLLM(MyLLM):
    def __init__(self, message_history: list, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.message_history = message_history

    def _preprocess(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        system_prompt = "あなたは優秀なAIアシスタントです"
        messages = [{"role": "system", "content": system_prompt}]

        # 過去の会話履歴を追加
        for prompt, response in self.message_history:
            messages.append({"role": "user", "content": prompt})
            messages.append({"role": "system", "content": response})

        user_prompt = f"{inputs['user_prompt']}"
        messages.append({"role": "user", "content": user_prompt})

        return messages

    def _postprocess(self, response) -> str:
        output = response.choices[0]["message"]["content"]
        return output


def chat_with_gpt(prompt, message_history):
    try:
        llm = ChatBotLLM(
            llm_settings={"temperature": 0.2, "max_tokens": 1000},
            model="gpt-3.5-turbo-1106",
            platform="openai",
            stream_verbose=False,  # TODO: stremaの実装
            message_history=message_history,
        )
        inputs = {"user_prompt": prompt}
        outputs = llm(inputs=inputs)
        message_history.append((prompt, outputs))
        return "", message_history

    except Exception as e:
        return f"Error: {e}"


with gr.Blocks() as app:
    gr.Markdown("# Chat with neoLLM")

    chatbot = gr.Chatbot()

    input = gr.Textbox(show_label=False, placeholder="Message neoLLM...")
    input.submit(fn=chat_with_gpt, inputs=[input, chatbot], outputs=[input, chatbot])

app.launch()

Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


PARENT
MyLLM(ChatBotLLM) ----------------------------------------------------------------------------------
[inputs]
{
  "user_prompt": "こんにちは"
}
[messages]
 system
  あなたは優秀なAIアシスタントです
 user
  こんにちは
 assistant
  こんにちは！お困りごとはありますか？お手伝いできることがあれば、なんでもお知らせくださいね。
[outputs]
"こんにちは！お困りごとはありますか？お手伝いできることがあれば、なんでもお知らせくださいね。"
[client_settings] -
[llm_settings] {'platform': 'openai', 'temperature': 0.2, 'max_tokens': 1000, 'model': 'gpt-3.5-turbo-1106'}
[metadata] 0.9s; 68(29+39)tokens; $6.8e-05; ¥0.0095
----------------------------------------------------------------------------------------------------
PARENT
MyLLM(ChatBotLLM) ----------------------------------------------------------------------------------
[inputs]
{
  "user_prompt": "私はkokiです"
}
[messages]
 system
  あなたは優秀なAIアシスタントです
 user
  こんにちは
 system
  こんにちは！お困りごとはありますか？お手伝いできることがあれば、なんでもお知らせくださいね。
 user
  私はkokiです
 assistant
  はじめまして、Kokiさん！何か質問やお手伝いできることはありますか？
[outputs]
"はじめまして、Kokiさん！何か質問やお手伝いできることはありますか？"
[client_settings] -
[llm_se

In [50]:
def chat_with_gpt(prompt, message_history):
    try:
        llm = ChatBotLLM(
            llm_settings={"temperature": 0.2, "max_tokens": 1000},
            model="gpt-3.5-turbo-1106",
            platform="openai",
            stream_verbose=False,  # TODO: stremaの実装
            message_history=message_history,
        )
        inputs = {"user_prompt": prompt}
        outputs = llm(inputs=inputs)
        message_history.append((prompt, outputs))
        return "", message_history

    except Exception as e:
        return f"Error: {e}"


def clear_history():
    return "", []


with gr.Blocks() as app:
    gr.Markdown("# Chat with neoLLM")

    chatbot = gr.Chatbot()
    input_text = gr.Textbox(show_label=False, placeholder="Message neoLLM...")

    input_text.submit(fn=chat_with_gpt, inputs=[
                      input_text, chatbot], outputs=[input_text, chatbot])
    submit_button = gr.Button("送信")
    submit_button.click(fn=chat_with_gpt, inputs=[
                        input_text, chatbot], outputs=[input_text, chatbot])

    clear_button = gr.Button("履歴をクリア")
    clear_button.click(fn=clear_history, inputs=[],
                       outputs=[input_text, chatbot])

app.launch()

Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.


PARENT
MyLLM(ChatBotLLM) ----------------------------------------------------------------------------------
[inputs]
{
  "user_prompt": "こんにちは"
}
[messages]
 system
  あなたは優秀なAIアシスタントです
 user
  こんにちは
 assistant
  こんにちは！お困りごとはありますか？何かお手伝いできることはありますか？
[outputs]
"こんにちは！お困りごとはありますか？何かお手伝いできることはありますか？"
[client_settings] -
[llm_settings] {'platform': 'openai', 'temperature': 0.2, 'max_tokens': 1000, 'model': 'gpt-3.5-turbo-1106'}
[metadata] 0.8s; 59(29+30)tokens; $5.9e-05; ¥0.0083
----------------------------------------------------------------------------------------------------


- 履歴を加味
- 送信ボタンとEnterボタンの追加

In [57]:
import gradio as gr


def chat_with_gpt(prompt, message_history):
    try:
        llm = ChatBotLLM(
            llm_settings={"temperature": 0.2, "max_tokens": 1000},
            model="gpt-3.5-turbo-1106",
            platform="openai",
            stream_verbose=False,  # TODO: stremaの実装
            message_history=message_history,
        )
        inputs = {"user_prompt": prompt}
        outputs = llm(inputs=inputs)
        message_history.append((prompt, outputs))
        return "", message_history

    except Exception as e:
        return f"Error: {e}"


def clear_history():
    return "", []


with gr.Blocks() as app:
    gr.Markdown("# Chat with neoLLM")

    chatbot = gr.Chatbot()
    input_text = gr.Textbox(show_label=False, placeholder="Message neoLLM...")

    input_text.submit(fn=chat_with_gpt, inputs=[
                      input_text, chatbot], outputs=[input_text, chatbot])
    submit_button = gr.Button("Send")
    submit_button.click(fn=chat_with_gpt, inputs=[
                        input_text, chatbot], outputs=[input_text, chatbot])

    clear_button = gr.Button("New Chat")
    clear_button.click(fn=clear_history, inputs=[],
                       outputs=[input_text, chatbot])

app.launch()

Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.


PARENT
MyLLM(ChatBotLLM) ----------------------------------------------------------------------------------
[inputs]
{
  "user_prompt": "hello"
}
[messages]
 system
  あなたは優秀なAIアシスタントです
 user
  hello
 assistant
  Hello! How can I assist you today?
[outputs]
"Hello! How can I assist you today?"
[client_settings] -
[llm_settings] {'platform': 'openai', 'temperature': 0.2, 'max_tokens': 1000, 'model': 'gpt-3.5-turbo-1106'}
[metadata] 0.6s; 38(29+9)tokens; $3.8e-05; ¥0.0053
----------------------------------------------------------------------------------------------------
PARENT
MyLLM(ChatBotLLM) ----------------------------------------------------------------------------------
[inputs]
{
  "user_prompt": "i am ishi"
}
[messages]
 system
  あなたは優秀なAIアシスタントです
 user
  i am ishi
 assistant
  Hello Ishi! How can I assist you today?
[outputs]
"Hello Ishi! How can I assist you today?"
[client_settings] -
[llm_settings] {'platform': 'openai', 'temperature': 0.2, 'max_tokens': 1000, 'model': 'gpt-3

## プルダウンでモデルを選択

In [61]:
import gradio as gr


def chat_with_gpt(prompt, model, message_history):
    try:
        llm_settings = {"temperature": 0.2, "max_tokens": 1000}
        llm = ChatBotLLM(
            llm_settings=llm_settings,
            model=model,  # プルダウンから選択されたモデルを使用
            platform="openai",
            stream_verbose=False,  # TODO: stremaの実装
            message_history=message_history,
        )
        inputs = {"user_prompt": prompt}
        outputs = llm(inputs=inputs)
        message_history.append((prompt, outputs))
        return "", message_history

    except Exception as e:
        return f"Error: {e}", message_history


def clear_history():
    return "", []


with gr.Blocks() as app:
    gr.Markdown("# Chat with neoLLM")

    model_dropdown = gr.Dropdown(
        label="Models",
        choices=[
            "gpt-3.5-turbo-0125",
            "gpt-4",
            "gpt-4-0125-preview",
            "gpt-4-turbo-preview"
        ],
        value="gpt-3.5-turbo-0125",
    )

    chatbot = gr.Chatbot()
    input_text = gr.Textbox(show_label=False, placeholder="Message neoLLM...")

    input_text.submit(fn=chat_with_gpt, inputs=[input_text, model_dropdown, chatbot], outputs=[input_text, chatbot])
    submit_button = gr.Button("Send")
    submit_button.click(fn=chat_with_gpt, inputs=[input_text, model_dropdown, chatbot], outputs=[input_text, chatbot])

    clear_button = gr.Button("+ New Chat")
    clear_button.click(fn=clear_history, inputs=[], outputs=[input_text, chatbot])

app.launch()

Running on local URL:  http://127.0.0.1:7900

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


## パラメータの調整

In [70]:
css = """
/* チャットボットのユーザーメッセージの背景色を変更 */
.message.user {
    background: #06c755 !important;
    color: white;
}

/* チャットボットのボットメッセージの背景色を変更 */
.message.bot {
    background: #007BFF !important;
    color: white;
}

/* ボタンのスタイルをカスタマイズ */
button {
    background-color: #06c755;
    color: white;
    border: none;
    padding: 10px 20px;
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 16px;
    margin: 4px 2px;
    transition-duration: 0.4s;
    cursor: pointer;
    border-radius: 5px;
}

button:hover {
    background-color: white;
    color: #06c755;
}

/* ドロップダウンのスタイルをカスタマイズ */
.select {
    width: 100%;
    padding: 12px 20px;
    margin: 8px 0;
    display: inline-block;
    border: 1px solid #ccc;
    border-radius: 4px;
    box-sizing: border-box;
}

/* スライダーのスタイルをカスタマイズ */
.slider {
    -webkit-appearance: none;
    width: 100%;
    height: 15px;
    border-radius: 5px;
    background: #06c755;
    outline: none;
    opacity: 0.7;
    -webkit-transition: .2s;
    transition: opacity .2s;
}

.slider:hover {
    opacity: 1;
}
"""

In [71]:
import gradio as gr


def chat_with_gpt(prompt, model, temperature, message_history):
    try:
        llm_settings = {"temperature": temperature, "max_tokens": 1000}  # Sliderからの入力を使用
        llm = ChatBotLLM(
            llm_settings=llm_settings,
            model=model,  # プルダウンから選択されたモデルを使用
            platform="openai",
            stream_verbose=False,  # TODO: stremaの実装
            message_history=message_history,
        )
        inputs = {"user_prompt": prompt}
        outputs = llm(inputs=inputs)
        message_history.append((prompt, outputs))
        return "", message_history

    except Exception as e:
        return f"Error: {e}", message_history


def clear_history():
    return "", []

with gr.Blocks(css=css) as app:
    # gr.Markdown("# Chat with neoLLM")

    model_dropdown = gr.Dropdown(
        label="Models",
        choices=["gpt-3.5-turbo-0125", "gpt-4", "gpt-4-0125-preview", "gpt-4-turbo-preview"],
        value="gpt-3.5-turbo-0125",
    )

    # temperatureの値を調整するためのSliderを追加
    temperature_slider = gr.Slider(minimum=0.0, maximum=1.0, step=0.01, value=0.2, label="Temperature")

    chatbot = gr.Chatbot()
    input_text = gr.Textbox(show_label=False, placeholder="Message neoLLM...")

    input_text.submit(
        fn=chat_with_gpt,
        inputs=[input_text, model_dropdown, temperature_slider, chatbot],
        outputs=[input_text, chatbot],
    )
    submit_button = gr.Button("Send")
    submit_button.click(
        fn=chat_with_gpt,
        inputs=[input_text, model_dropdown, temperature_slider, chatbot],
        outputs=[input_text, chatbot],
    )

    clear_button = gr.Button("+ New Chat")
    clear_button.click(fn=clear_history, inputs=[], outputs=[input_text, chatbot])

app.launch()

Running on local URL:  http://127.0.0.1:7905

To create a public link, set `share=True` in `launch()`.
